## Package Imports

In [ ]:
!pip install langchain_neo4j -q
!pip install python-dotenv -q
!pip install langchain_google_genai -q
!pip install langgraph -q
!pip install langchain-community -q
!pip install faiss-cpu -q

In [23]:
import asyncio
import pandas as pd
import json
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from operator import add
from typing import Annotated, List
from typing_extensions import TypedDict
from functools import lru_cache
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

In [3]:
from google.colab import userdata
os.environ["NEO4J_URI"] = userdata.get("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = userdata.get("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = userdata.get("NEO4J_PASSWORD")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

In [4]:
assert os.getenv("GEMINI_API_KEY"), "GEMINI_API_KEY not set in environment"
assert os.getenv("NEO4J_URI"), "NEO4J_URI not set in environment"
assert os.getenv("NEO4J_USERNAME"), "NEO4J_USERNAME not set in environment"
assert os.getenv("NEO4J_PASSWORD"), "NEO4J_PASSWORD not set in environment"

In [18]:
import nest_asyncio
nest_asyncio.apply()

## Neo4j Driver

In [5]:
enhanced_graph = Neo4jGraph(
  driver_config={
        'connection_acquisition_timeout': 30,
        'max_connection_pool_size': 50,
        'keep_alive': True,
        'max_connection_lifetime': 1800,
        'connection_timeout': 10,
  },
)

In [6]:
graph_schema = enhanced_graph.schema
graph_structured_schema = enhanced_graph.structured_schema

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=2))
async def safe_query(query, params=None):
    enhanced_graph._driver.verify_connectivity()

    if params:
      return enhanced_graph.query(
        query=query,
        params=params
      )
    else:
      return enhanced_graph.query(
        query=query
      )


## LLM Declaration

In [7]:
LLM_MODEL = "gemini-2.5-flash-preview-04-17"

llm = ChatGoogleGenerativeAI(
    model=LLM_MODEL,
    api_key=os.environ["GEMINI_API_KEY"],
    temperature=0,  # Deterministic output
    top_p=1.0       # Full probability distribution
)

EMBEDDING_MODEL = "models/embedding-001"

embedding_model = GoogleGenerativeAIEmbeddings(
      model=EMBEDDING_MODEL, task_type="semantic_similarity",
      google_api_key=os.getenv("GEMINI_API_KEY"),
    )

## State Definition

In [8]:
class InputState(TypedDict):
  question: str
  passing_threshold: float

class OverallState(TypedDict):
  question: str
  passing_threshold: float
  next_action: str
  cypher_statement: str
  cypher_errors: List[str]
  database_records: List[dict]
  steps: Annotated[List[str], add]

class OutputState(TypedDict):
  answer: str
  cypher_statement: str
  steps: List[str]
  database_records: List[dict]

## Guardrail

In [79]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal


guardrails_system = """
As an intelligent assistant, your primary objective is to decision strictly on the following basis:

1. If the question is related to food ordering/listing then output "zomato".

3. If the question is not related to food or restaurant then output "end".
.
To make this decision, assess the content of the question and determine if it refers to any food, food order, detail about food or restaurant or details about restaurant,
or related topics. Provide only the specified output: "zomato" or "end".
"""

guardrails_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            guardrails_system,
        ),
        (
            "human",
            ("{question}"),
        )
    ]
)

class GuardrailsOutput(BaseModel):
  decision: Literal["food", "restaurant", "end"] = Field(
      description="Decision on whether the question is related to foods or restaurant or anything else"
  )

guardrails_chain = guardrails_prompt | llm.with_structured_output(GuardrailsOutput)

async def guardrails(state: InputState) -> OverallState:
    """
    Decides if the question is related to foods or restaurant or not.
    """

    guardrails_output = await guardrails_chain.ainvoke({"question": state.get("question")})
    database_records = None

    if guardrails_output.decision == "end":
      database_records = "This questions is not about food (ordering/detail) or their related. Therefore I cannot answer this question."

    return {
        "next_action": guardrails_output.decision,
        "database_records": database_records,
        "steps": ["guardrail"],
    }

## Parameter Based Agent



### List of examples for entities

In [83]:
examples_for_entity_extraction = [
      {
        "question": "Hey, I want to spicy saucey chessy pasta with tacos",
        "order_info": '''[
              {
                "food_name": "pasta",
                "flavour": "spicy saucey chessy",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "tacos",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Hey, show me the deal for bestselling spicy kadai paneer and mango lassi with butter garlic naan, from the restaurant that has atleast 2.0 rating.",
        "order_info": '''[
              {
                "food_name": "kadai paneer",
                "flavour": "spicy",
                "bestseller": "true",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "2.0",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "lassi",
                "flavour": "mango",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "2.0",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "naan",
                "flavour": "butter garlic",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "2.0",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "what's the veg menu of KFC?",
        "order_info": '''[
              {
                "food_name": "",
                "flavour": "",
                "bestseller": "false",
                "type_": "veg",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "KFC", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "What are the options available for sweet corn pizza available at dominos?",
        "order_info": '''[
              {
                "food_name": "pizza",
                "flavour": "sweet corn",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "dominos", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "I want to eat masala dosa with plain lassi from restaurant which serves south indian as well as north indian dessert food with good rating.",
        "order_info": '''[
              {
                "food_name": "masala dosa",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "south indian, north indian dessert",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0,
                "order_filter": {"restaurant_rating_filter": "DESC"}
              },
              {
                "food_name": "lassi",
                "flavour": "plain",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "south indian, north indian dessert",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0,
                "order_filter": {"restaurant_rating_filter": "DESC"}
              }
          ]'''
      },
      {
        "question": "Hey, I want to order veg indi tandoori pizza with spicy cheese dip from dominos and 3 crispy chicken burger from McDonald's",
        "order_info": '''[
              {
                "food_name": "indi tandoori pizza",
                "flavour": "",
                "bestseller": "false",
                "type_": "veg",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "dominos", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "cheese dip",
                "flavour": "spicy",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "dominos", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "chicken burger",
                "flavour": "crispy",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 3,
                "restaurant_name_pair": [("restaurant_name": "McDonald's", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Show me a deal that include butter chicken from burj kebab and 2 stuffed naan which has rating atleast 4.0 from any other restaurant, ensure that each naan won't cost more than 100.",
        "order_info": '''[
              {
                "food_name": "butter chicken",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "burj kebab", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              },
              {
                "food_name": "stuffed naan",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "4.0",
                "food_price": 100.0,
                "quantity": 2,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Are there restaurants available near rajiv chowk that servers italian food?",
        "order_info": '''[
              {
                "food_name": "",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "italian",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "rajiv chowk",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Hey, can you confirm which restaurant has 1143595672 as its phone number?",
        "order_info": '''[
              {
                "food_name": "",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "1143595672",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Hey, do you have any option for turkish dessert?",
        "order_info": '''[
              {
                "food_name": "dessert",
                "flavour": "turkish",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "Do you have options for sundae ice cream, show me only top 20 highest rated, and also don't show me an options from baker's ice cream?",
        "order_info": '''[
              {
                "food_name": "ice cream",
                "flavour": "sundae",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "baker's ice cream", "condition": False)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 20,
                "order_filter": {"food_rating_filter": "DESC"}
              }
          ]'''
      },
      {
        "question": "what all restaurant are there who servers turkish food, show me only top 5 highest rated?",
        "order_info": '''[
              {
                "food_name": "",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "turkish",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 5,
                "order_filter": {"restaurant_rating_filter": "DESC"}
              }
          ]'''
      },
      {
        "question": "Hey, show me a chinese meal under 300, don't show me from chopstick and ching chinese.",
        "order_info": '''[
              {
                "food_name": "",
                "flavour": "chinese",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 300.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "chopstick", "condition": False), ("restaurant_name": "ching chinese", "condition": False)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "show me an option for most expensive cheese burger from KFC, not from McDonald",
        "order_info": '''[
              {
                "food_name": "burger",
                "flavour": "cheese",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "KFC", "condition": True), ("restaurant_name": "McDonald", "condition": False)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 1,
                "order_filter": {"food_price_filter": "DESC"}
              }
          ]'''
      },
      {
        "question": "Hey, do KfC has options for chicken meal under 400.",
        "order_info": '''[
              {
                "food_name": "chicken meal",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 400.0,
                "quantity": 1,
                "restaurant_name_pair": [("restaurant_name": "KFC", "condition": True)],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": "",
                "limit": 0
              }
          ]'''
      },
      {
        "question": "My order: 1 - Kadai Paneer, 1 - Mix Veg, 6 - Roti or tandoori roti",
        "order_info": '''[
              {
                "food_name": "Kadai Paneer",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": ""
              },
              {
                "food_name": "Mix Veg",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 1,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": ""
              },
              {
                "food_name": "Roti or tandoori roti",
                "flavour": "",
                "bestseller": "false",
                "type_": "not_mentioned",
                "food_rating": "not_available",
                "food_price": 0.0,
                "quantity": 6,
                "restaurant_name_pair": [],
                "restaurant_deliverables": "",
                "restaurant_rating": "not_available",
                "restaurant_phone_number": "",
                "restaurant_address": ""
              }
          ]'''
      }
]

In [84]:
with open("examples_for_entity_extraction.json", "w") as f:
    json.dump(examples_for_entity_extraction, f, indent=4)

### Order Entities

In [85]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, field_validator
from typing import List, Literal, Union, Tuple, Optional
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import os
import json

documents = [
    Document(page_content=ex["question"])
    for ex in examples_for_entity_extraction
]

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

entity_example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples_for_entity_extraction,
    embeddings = embedding_model,
    vectorstore_cls=vectorstore,
    k=5,
    input_keys=["question"],
)

class RestaurantNamePair(BaseModel):
  restaurant_name: str
  condition: bool

class OrderFiltering(BaseModel):
  food_rating_filter: Optional[Literal["ASC", "DESC"]]=None
  food_price_filter: Optional[Literal["ASC", "DESC"]]=None
  restaurant_rating_filter: Optional[Literal["ASC", "DESC"]]=None

# Extract entities from text
class EntityInfoItem(BaseModel):
    """Identifying information about entities."""
    food_name: str = Field(
        description="""
        It represents the food (dish) name only, if no name is mentioned consider the default value only.

        IMPORTANT: It's not a common noun remember that, words like "food", "dish" or their synonyms are not valid values.

        For example:
        question: "Hey, what's the worst rated dish you have?"
        food_name: ""

        question: "Hey, I want to eat some spicy food?"
        food_name: ""

        question: "Are they any option for mango shakes?"
        food_name: "shakes"

        question: "Hello, I'm looking for a meal of pizza."
        food_name: "pizza"
        """,
        default=""
    )

    flavour: str = Field(
        description="""
        It represents the flavour of the food(dish) described in the question.
        """,
        default=""
    )
    bestseller: Literal["true", "false"] = Field(
        ...,
        description="""
        Whether the food is supposed to be a bestseller or not, it's value is "true" only when the keyword like bestseller, best selling item, popular, recommended, is present.
        """,
    )

    type_: Literal["veg", "non-veg", "egg", "not_mentioned"] = Field(
        description="""
        If the food type is mentioned from any one of the following types: veg, non-veg or egg. If nothing is present about the food type then consider its value equal to "not_mentioned".
        Never consider any other value for the food type. Only permissible values are "veg", "non_veg", "egg" and "not_mentioned".
        """,
        default="not_mentioned"
    )

    food_rating: Union[float, Literal["not_available"]] = Field(
        ...,
        description="""
        Represents the food_rating of the food. Unless any "numeric value" is present then rating is equal to "not_available".
        """,
    )

    food_price: float = Field(
        description="""
        Represents the price of the food.
        """,
        default=0.0
    )

    quantity: int = Field(
        description="""
        It represents the quantity of food mentiioned in the question, if nothing is mentioned consider the default value.

        For example: if question is "Hey, I want to order 2 scoops of ice cream" then quantity is equal to 2.

        """,
        default = 1
    )

    restaurant_name_pair: List[RestaurantNamePair] = Field(
        description="""
        A list of (restaurant_name, condition) pairs where:
        - `restaurant_name` is a string representing the name.
        - `condition` is a boolean: True means the name should be matched using CONTAINS,
          False means the name should NOT be matched using CONTAINS.

        Example: [("restaurant_name": "KFC", "condition": True), ("restaurant_name": "McDonald", "condition": False)]

        VERY IMPORTANT: Don't change anything with characters of the name, keep each character
        same as it is, even the apostrophe character, comma, quotes, etc.
        """,
        default=[]
    )


    restaurant_deliverables: str = Field(
        description="""
        It represent the kind of food is expected from the restaurant like dessert, sweet, north india, south indian, italian food, turkish, spanish food, chinese food, juices, shakes, mountain food, drinks etc.
        If nothing is expected from the restaurant then consider the default value.
        """,
        default=""
    )

    restaurant_rating: Union[float, Literal["not_available"]] = Field(
        ...,
        description="""
        Represents the delivery_rating of the restaurant. Unless any "numeric value" is present then rating is equal to "not_available".
        """,
    )

    restaurant_phone_number: str = Field(
        description="""
        Represent the phone_number of a restaurant if the input question contains it.
        """,
        default=""
    )

    restaurant_address: str = Field(
        description="""
        Represent the address of a restaurant if the input question contains it.
        """,
        default = ""
    )

    limit: int = Field(
        description="""
        It represent the number of records user demanded in the question.

        Examples:
        1. question: "Hey, show me the most expensive dish", limit: 1
        2. question: "Show me the top 10 cheapest food option", limit: 10
        3. question: "show me the cheapest option for pizza at zomato.", limit: 1

        IMPORTANT: Unless keywords like most expensive/cheapest, top 5 or top n (n could be any integer), most pricest, etc are present consider the default value 0 only.
        """,
        default=0
    )

    order_filter: Optional[OrderFiltering] = Field(
        description="""
        It represents the way/order in which user wants his/her records to be arranged based on the context of user question.

        Examples:
        1. question: "Hey, show me the most expensive dish", order_filter: {"food_price_filter": "DESC"}
        2. question: "Show me the top 10 cheapest food option from high rating restaurants", order_filter: {"restaurant_rating_filter": "DESC", "food_price_filter": "ASC"}
        3. question: "show me an option for kadai paneer with good rating", order_filter: {"food_rating_filter": "DESC"}

        IMPORTANT: This is an optional field consider it only when it is reflected from the user question that the user wants his/her records to arranged in a specific way.
        """
    )

    @field_validator("food_rating", "restaurant_rating", mode="before")
    @classmethod
    def parse_rating(cls, value):
        try:
            return str(float(value))
        except (ValueError, TypeError):
            return "not_available"

    @field_validator('restaurant_name_pair')
    @classmethod
    def validate_pairs(cls, v):
        if not all(isinstance(item, RestaurantNamePair) for item in v):
            raise ValueError("Each item in restaurant_name_pair must be a RestaurantNamePair object")
        return v

class Entities(BaseModel):
  order_info: List[EntityInfoItem] = Field(
        default = [],
        description="""
        It represents the list of dictionaries of type FoodInfoItem. The number of dictionaries in the list is always equal to the number of food(dish) names present in the question.
        """
  )

entity_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You're an intelligent assistant who is expert in understanding the requirement of the user only related to food ordering.
            """
        ),
        (
            "human",
            """
            Use the given format to extract information from the following, strictly be consistent with your response don't change it for the same question."

            "input: {question}",

            Below are a list of examples for a reference.
            {examples}
            """
        ),
    ]
)

entity_chain = entity_prompt | llm.with_structured_output(Entities)

async def get_entities(question):
    NL = '\n'
    fewshot_examples = (NL*2).join(
        [
            f"Question: {el['question']}{NL}OrderInfo:{el['order_info']}"
            for el in entity_example_selector.select_examples(
                {"question": question}
            )
        ]
    )

    extracted_entities = await entity_chain.ainvoke({"question": question, "examples": fewshot_examples})
    return extracted_entities

In [88]:
# await get_entities('Show me a good deal for a bestselling dal fry with roti under 300 from restaurant which has good delivery rating')

### Food Vector Index

In [13]:
from langchain_neo4j import Neo4jVector

food_vector_index = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    index_name="food_embedding_index",             # vector index name
    keyword_index_name="food_fulltext_index",      # explicitly pass keyword index name
    search_type="hybrid",                          # hybrid search: vector + keyword
    node_label="Food",
    text_node_properties=["price", "bestseller", "name", "type", "desc", "rating", "category", "restaurant_name", "id"],
    embedding_node_property="embedding"
)

### Generate Parameter Based Cypher Query

### Generate Parameter Based Records

In [24]:
import re
import os
import json
import numpy as np
import pandas as pd
from functools import reduce
import time

In [14]:
entity_cypher_map = {
    'delivery_rating': '''
      r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available'
      AND toFloatOrNull(r.delivery_rating) >= $delivery_rating
      ''',

    'phone_number': '''
      r.phone_no CONTAINS $phone_number
      ''',

    'address': '''
      toLower(r.address) CONTAINS $address
      ''',

    'deliverables': '''
      CALL db.index.fulltext.queryNodes('restaurant_deliverables_fulltext_index', $deliverables) YIELD node AS r, score AS restaurant_score
      ''',

    'name': '''
      ALL(pair IN $name WHERE
          (pair[1] = true AND toLower(r.name) CONTAINS toLower(pair[0]))
          OR
          (pair[1] = false AND NOT toLower(r.name) CONTAINS toLower(pair[0]))
      )
      ''',

    'food_scores': '''
      UNWIND $food_scores AS fs
      ''',

    'bestseller': '''
      f.bestseller = true
      ''',

    'type': '''
      f.type = $type
      ''',

    'food_rating': '''
      (f.rating IS NOT NULL AND f.rating <> 'not_available')
      AND toFloatOrNull(f.rating) >= $food_rating
      ''',

    'price': '''
      f.price <= $price + $tolerance
      '''
}

In [15]:
async def get_search_query_and_params(entity: EntityInfoItem, tolerance: int)->dict:
    search_query = ""
    params = {}

    '''Restaurant_parameters'''
    if entity.restaurant_rating != "not_available":
      params["delivery_rating"] = entity.restaurant_rating

    if entity.restaurant_phone_number:
      params["phone_number"] = entity.restaurant_phone_number

    if entity.restaurant_address:
      params["address"] = entity.restaurant_address.lower()

    if entity.restaurant_deliverables:
      params["deliverables"] = entity.restaurant_deliverables.lower()

    '''Food_parameters'''

    if entity.flavour:
      search_query += entity.flavour.lower()

    if entity.food_name:
      food_name = entity.food_name.lower()

      if food_name not in ['food', 'dish']:
        if search_query != "":
          search_query += " " + food_name
        else:
          search_query += food_name

    if entity.bestseller=="true":
      if search_query != "":
        search_query += ", " + "bestseller: true"
      params["bestseller"] = "true"

    if entity.type_!="not_mentioned":
      if search_query != "":
        search_query += ", " + "type: " + entity.type_
      params["type"] = entity.type_

    if entity.food_rating!="not_available":
      if search_query != "":
        search_query += ", " + "rating: " + str(entity.food_rating)
      params["food_rating"] = entity.food_rating

    if entity.food_price!=0.0:
      if search_query != "":
        search_query += ", " + "price: " + str(entity.food_price)
      params["price"] = entity.food_price
      params["tolerance"] = tolerance

    if entity.restaurant_name_pair:
      restaurant_name_pair = entity.restaurant_name_pair

      restaurant_name_pair_list = []

      for pair in restaurant_name_pair:
        restaurant_name = pair.restaurant_name.lower()
        condition = pair.condition
        if restaurant_name != "zomato":
          if search_query != "" and condition:
            search_query += ", " + "restaurant_name: " + pair.restaurant_name
          restaurant_name_pair_list.append([restaurant_name, condition])

      if restaurant_name_pair_list:
        params["name"] = restaurant_name_pair_list


    if entity.limit!=0:
      params["limit"] = entity.limit

    if entity.order_filter:
      if entity.order_filter.food_rating_filter:
        params['food_rating_filter'] = entity.order_filter.food_rating_filter
      if entity.order_filter.food_price_filter:
        params['food_price_filter'] = entity.order_filter.food_price_filter
      if entity.order_filter.restaurant_rating_filter:
        params['restaurant_rating_filter'] = entity.order_filter.restaurant_rating_filter

    return search_query, params

In [16]:
async def get_food_scores(search_query: str, passing_threshold:float):
  food_scores = []
  hybrid_search_output = food_vector_index.similarity_search_with_score(query=search_query, k=1000)

  for doc, score in hybrid_search_output:
      if score < passing_threshold:
          break
      match = re.search(r'id:\s*([a-f0-9\-]+)', doc.page_content)
      if match:
          node_id = match.group(1)
          food_scores.append({"id": node_id, "score": score})

  return food_scores


In [49]:
list_of_keys = ['restaurant', 'restaurant_score', 'zomato_page', 'restaurant_image_url', 'delivery_rating', 'dining_rating', 'deliverables', 'phone_number',
                'address', 'food_name', 'food_type', 'bestseller', 'price', 'quantity', 'food_rating', 'description', 'food_image_url', 'similarity_score']

async def sort_final_df(final_df):
    # 1) Count non‑null restaurant_* families
    restaurant_cols = [c for c in final_df.columns if re.match(r'restaurant_\d+$', c)]
    restaurant_count = final_df[restaurant_cols].notna().sum(axis=1)

    # 2) Compute average similarity_score_* per row (if any such columns exist)
    sim_cols = [c for c in final_df.columns if re.match(r'similarity_score_\d+$', c)]
    if sim_cols:
        sim_mean = final_df[sim_cols].mean(axis=1, skipna=True)
    else:
        # no columns → all zeros so they sort to bottom
        sim_mean = pd.Series(0, index=final_df.index)

    # 3) Compute average restaurant_score_* per row (if any exist)
    score_cols = [c for c in final_df.columns if re.match(r'restaurant_score_\d+$', c)]
    if score_cols:
        score_mean = final_df[score_cols].mean(axis=1, skipna=True)
    else:
        score_mean = pd.Series(0, index=final_df.index)

    # 4) Attach these as temporary columns (optional) or sort via the Series directly
    final_df.loc[:, '_rest_count'] = restaurant_count
    final_df.loc[:, '_sim_mean']   = sim_mean
    final_df.loc[:, '_score_mean'] = score_mean

    # 5) Sort by the three keys, all descending
    final_df.sort_values(
        ['_rest_count', '_sim_mean', '_score_mean'],
        ascending=[False, False, False],
        inplace=True
    )

    # 6) Drop the helpers
    final_df.drop(columns=['_rest_count', '_sim_mean', '_score_mean'], inplace=True)
    final_df.reset_index(inplace=True, drop=True)

    return final_df

async def prepare_db_records(list_of_dataframe, params, n):
  start = time.time()

  output = []

  final_df = reduce(lambda left, right: pd.merge(left, right, on='restaurant_id', how='outer'), list_of_dataframe)

  final_df = final_df.drop_duplicates().copy()

  '''Arrange rows in final_df in descending order of number of dishes found..'''
  # restaurant_cols = [f'restaurant_{sufx}' for sufx in range(1, n+1)]
  # non_null_counts = final_df[restaurant_cols].notna().sum(axis=1)
  # final_df = final_df.loc[non_null_counts.sort_values(ascending=False).index]
  # final_df.reset_index(inplace=True, drop=True)

  print("Time spent in merging dataframe: ", round(time.time()-start, 2))
  print("-----"*12)
  start = time.time()

  final_df = await sort_final_df(final_df)

  print("Time spent in sorting dataframe: ", round(time.time()-start, 2))
  print("-----"*12)

  limit = params.get('limit', 1000)

  x = set(params.keys()) == set(['quantity', 'limit', 'food_price_filter'])
  x = x or set(params.keys()) == set(['quantity', 'limit', 'food_rating_filter'])
  x = x or set(params.keys()) == set(['quantity', 'limit', 'restaurant_rating_filter'])

  if not x and final_df.shape[0] > limit:
    final_df = final_df.head(limit)


  start = time.time()
  for _, row in final_df.iterrows():
    total_cost = 0
    avg_similarity_score = []
    deal = []
    for i in range(1, n+1):
      if pd.notna(row[f'restaurant_{i}']):
        deal_item = {}

        if f'price_{i}' in row and f'quantity_{i}' in row and row[f'quantity_{i}']:
          total_cost += row[f'price_{i}']*int(row[f'quantity_{i}'])

        if f'similarity_score_{i}' in row:
          avg_similarity_score.append(row[f'similarity_score_{i}'])
        elif f'restaurant_score_{i}' in row:
          avg_similarity_score.append(row[f'restaurant_score_{i}'])

        deal_item = {key: row[f'{key}_{i}'] for key in list_of_keys if f'{key}_{i}' in row}
        deal.append(deal_item)

    if avg_similarity_score:
      avg_similarity_score = round(float(np.mean(avg_similarity_score)), 3)


    data = {'deal': deal}
    if total_cost != 0:
        data['total_cost'] = total_cost
    if avg_similarity_score:
        data['avg_similarity_score'] = avg_similarity_score

    output.append(data)

  filter_flag = False

  if 'food_price_filter' in params:
    try:
      price_pfx = 1
      if params['food_price_filter'] == 'DESC':
        price_pfx = -1

      output.sort(
          key=lambda x: (-1 * len(x['deal']), -1 * x['avg_similarity_score'], price_pfx * x['total_cost']) if 'avg_similarity_score' in x else (-1 * len(x['deal']), price_pfx * x['total_cost'])
      )

      filter_flag = True
    except:
      pass

  elif 'food_rating_filter' in params:
    try:
      rating_pfx = 1
      default_value = 1e3
      if params['food_rating_filter'] == 'DESC':
        rating_pfx = -1
        default_value = 0

      output.sort(
          key = lambda x: (-1*len(x['deal']), -1*x['avg_similarity_score'], rating_pfx*(x['deal'][0]['food_rating'], default_value)) if 'avg_similarity_score' in x else (-1*len(x['deal']), rating_pfx*(x['deal'][0]['food_rating'], default_value))
      )

      filter_flag = True
    except:
      pass

  elif 'restaurant_rating_filter' in params:
    try:
      restaurant_rating_rating_pfx = 1
      default_value = 1e3
      if params['restaurant_rating_filter'] == 'DESC':
        restaurant_rating_rating_pfx = -1
        default_value = 0

      output.sort(
          key = lambda x: (-1*len(x['deal']), -1*x['avg_similarity_score'], restaurant_rating_rating_pfx*(x['deal'][0]['delivery_rating'], default_value)) if 'avg_similarity_score' in x else (-1*len(x['deal']), restaurant_rating_rating_pfx*(x['deal'][0]['delivery_rating'], default_value)),
      )
      filter_flag = True
    except:
      pass


  print("filter_flag: ", filter_flag)

  if not filter_flag:
    output.sort(
        key = lambda x: (-1*len(x['deal']), -1*x['avg_similarity_score']) if 'avg_similarity_score' in x else (-1*len(x['deal']))
    )


  del final_df

  if len(output)!=limit:
    output = output[:limit]

  print("Time spent in final output creation: ", round(time.time()-start, 2))
  print("-----"*12)

  return output


In [63]:
async def build_cypher_query(entity, search_query, params):
  cypher_query = '''
  '''
  x = set(params.keys()) == set(['limit', 'food_price_filter'])
  x = x or set(params.keys()) == set(['limit', 'food_rating_filter'])

  if entity.quantity and (x or search_query or any(key in ['price', 'type', 'food_rating', 'bestseller'] for key in params.keys())):
    params["quantity"] = entity.quantity

  print("params: ", params)
  print("-----"*12)

  if 'deliverables' in params:
    cypher_query += entity_cypher_map['deliverables'] + ' '

  if 'food_scores' in params:
    cypher_query += entity_cypher_map['food_scores'] + ' '
    cypher_query += 'MATCH (r:Restaurant)-[:DELIVERS]->(f:Food {id: fs.id})' + ' '
  elif 'quantity' in params:
    cypher_query += 'MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)' + ' '
  else:
    cypher_query += 'MATCH (r:Restaurant)' + ' '

  where_cypher_query = ''''''

  list_of_param_keys_not_for_where_clause = ['deliverables', 'food_scores', 'quantity', 'tolerance', 'food_rating_filter', 'food_price_filter', 'restaurant_rating_filter', 'limit']

  for key in params.keys():
    if key not in list_of_param_keys_not_for_where_clause:
      if where_cypher_query:
        where_cypher_query += 'AND ' + entity_cypher_map[key] + ' '
      else:
        where_cypher_query += 'WHERE ' + entity_cypher_map[key] + ' '

  cypher_query += where_cypher_query + ' '

  return_cypher_query = '''
  '''
  if 'food_scores' in params and 'deliverables' in params:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url  AS zomato_page,
                              r.delivery_rating AS delivery_rating,
                              f.name AS food_name,
                              f.bestseller AS bestseller,
                              f.price AS price,
                              f.type AS food_type,
                              coalesce($quantity, 1) AS quantity,
                              CASE
                              WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
                              ELSE NULL
                              END AS food_rating,
                              f.desc AS description,
                              f.image_url AS food_image_url,
                              fs.score AS similarity_score,
                              restaurant_score
                            '''
  elif 'quantity' in params and 'deliverables' in params:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url  AS zomato_page,
                              r.delivery_rating AS delivery_rating,
                              f.name AS food_name,
                              f.bestseller AS bestseller,
                              f.price AS price,
                              f.type AS food_type,
                              coalesce($quantity, 1) AS quantity,
                              CASE
                              WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
                              ELSE NULL
                              END AS food_rating,
                              f.desc AS description,
                              f.image_url AS food_image_url,
                              restaurant_score
                            '''

  elif 'food_scores' in params:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url AS zomato_page,
                              r.delivery_rating AS delivery_rating,
                              f.name AS food_name,
                              f.bestseller AS bestseller,
                              f.price AS price,
                              f.type AS food_type,
                              coalesce($quantity, 1) AS quantity,
                              CASE
                                WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
                                ELSE NULL
                              END AS food_rating,
                              f.desc AS description,
                              f.image_url AS food_image_url,
                              fs.score AS similarity_score
                            '''
  elif 'quantity' in params:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url AS zomato_page,
                              r.delivery_rating AS delivery_rating,
                              f.name AS food_name,
                              f.bestseller AS bestseller,
                              f.price AS price,
                              f.type AS food_type,
                              coalesce($quantity, 1) AS quantity,
                              CASE
                                WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
                                ELSE NULL
                              END AS food_rating,
                              f.desc AS description,
                              f.image_url AS food_image_url
                            '''
  elif 'deliverables' in params:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url AS zomato_page,
                              r.image_url AS restaurant_image_url,
                              CASE
                                WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
                                ELSE NULL
                              END AS delivery_rating,
                              CASE
                                WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
                              ELSE NULL
                              END AS dining_rating,
                              r.deliverables AS deliverables,
                              r.phone_no AS phone_number,
                              r.address AS address,
                              restaurant_score
                            '''
  else:
    return_cypher_query = '''RETURN r.id AS restaurant_id,
                              r.name AS restaurant,
                              r.url AS zomato_page,
                              r.image_url AS restaurant_image_url,
                              CASE
                                WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
                                ELSE NULL
                              END AS delivery_rating,
                              CASE
                                WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
                              ELSE NULL
                              END AS dining_rating,
                              r.deliverables AS deliverables,
                              r.phone_no AS phone_number,
                              r.address AS address
                            '''

  cypher_query += return_cypher_query

  return cypher_query


In [122]:
async def process_entity(entity, tolerance, passing_threshold, index):
    search_query, params = await get_search_query_and_params(entity, tolerance)

    food_scores = []
    if search_query:
        food_scores = await get_food_scores(search_query, passing_threshold)

    if food_scores:
        params["food_scores"] = food_scores

    # Build Cypher query (same as your existing logic)
    cypher_query = await build_cypher_query(entity, search_query, params)

    result = await safe_query(query=cypher_query, params=params)

    if result:
        temp_df = pd.DataFrame(result)
        temp_df.rename(columns=lambda x: f"{x}_{index}" if x != 'restaurant_id' else x, inplace=True)
        return (temp_df, params)

    return None

async def generate_database_records(state):
    entities = await get_entities(question=state.get('question'))
    tolerance = 10
    passing_threshold = state.get('passing_threshold', 0.98)

    tasks = [
        process_entity(entity, tolerance, passing_threshold, i + 1)
        for i, entity in enumerate(entities.order_info)
    ]

    results = await asyncio.gather(*tasks)

    list_of_dataframe = [res[0] for res in results if res is not None]
    if not list_of_dataframe:
      return {
          'next_action': 'generate_cypher',
          'steps': ['extract_entities', 'no_parameter_found', 'go_for_general_query_agent']
      }

    params = results[-1][1] if results[-1] is not None else {}

    output = await prepare_db_records(list_of_dataframe, params=params, n=len(list_of_dataframe))
    return {
        'next_action': 'generate_final_answer',
        'database_records': output,
        'steps': ['extract_entities', 'generate_parameter_based_cypher_query', 'execute_queries', 'generate_database_records']
    }


In [39]:
import asyncio
questio = "My order: 1 - Kadai Paneer, 1 - Mix Veg, 6 - Roti or tandoori roti"
state = OverallState(question=questio, passing_threshold=0.95)
result = asyncio.run(generate_database_records(state))

params:  {'food_scores': [{'id': '6b135647-fa56-4956-bc09-f3b6655b8ed8', 'score': 1.0}, {'id': '0ce8df2c-fa8a-4688-baab-0c496cc436e9', 'score': 1.0}, {'id': '8cee24af-fbf0-4237-b447-89fcf9240716', 'score': 0.998344004838676}, {'id': 'f9fc12cd-06af-42c4-a27f-018246ddfb67', 'score': 0.998344004838676}, {'id': '7467601a-3af4-4cc4-acce-d672bc3bf34e', 'score': 0.9923483221056584}, {'id': '8bf06446-df62-439b-bb30-0fef93954894', 'score': 0.992013056082758}, {'id': '729f551f-1e8f-4e1e-a600-a5038e44f9d3', 'score': 0.992013056082758}, {'id': 'a65aac82-7eb9-4941-bb77-4a26828adddd', 'score': 0.991686132604751}, {'id': 'c228f847-45a3-4b82-bff9-8a6605e5020e', 'score': 0.991686132604751}, {'id': 'dbcae363-97c6-4203-9f36-dc874a195421', 'score': 0.9910437566479655}, {'id': '776dc798-c295-4c0b-8756-284ee91c7cd9', 'score': 0.9901172127557512}, {'id': '84db9158-8018-4e30-ab1c-fe2aea9480e8', 'score': 0.9899159488601998}, {'id': 'ef131708-d8c1-4a6f-9144-71f89c652d4c', 'score': 0.988175746136359}, {'id': '8d

## General Query Agent

### Examples for fewshot technique

In [26]:
# @title
examples = [
  {
    "question": "Show me some options for paneer dishes from restaurants that have atleast 1 rating.",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE toLower(f.name) CONTAINS " paneer "
    AND r.delivery_rating <> "not_available"
    AND toFloat(r.delivery_rating) > 1
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "I’m looking for veg pasta options that cost less than 300, ordering for 4 persons. Can you help?",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE toLower(f.name) CONTAINS " pasta "
    AND toLower(f.type) = "veg"
    AND f.price < 300
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      4 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "I’m looking for non-veg pasta options that cost less than 300. Can you help?",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE toLower(f.name) CONTAINS " pasta "
    AND toLower(f.type) = "non-veg"
    AND f.price < 300
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "Show me where I can get ras malai.",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE toLower(f.name) CONTAINS "ras malai"
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "I want to eat some Spanish food. What options do I have?",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE toLower(c.name) CONTAINS " spanish " OR toLower(f.desc) CONTAINS " spanish "
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "What’s the most expensive dish you have? Show me the top 10.",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
    OPTIONAL MATCH (f)-[:COMES_UNDER]->(c:Category)
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    LIMIT 10
    '''
  },
  {
    "question": "What’s the cheapest options you have? Show me the top 10.",
    "query":'''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
    OPTIONAL MATCH (f)-[:COMES_UNDER]->(c:Category)
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price ASC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    LIMIT 10
    '''
  },
  {
    "question": "Any Indian sweets with kaju, like burfi or katli, available nearby, need 3 units?",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
    WHERE (toLower(f.name) CONTAINS "kaju" OR toLower(f.desc) CONTAINS "kaju") AND (toLower(c.name) CONTAINS "dessert" OR toLower(c.name) CONTAINS "sweet")
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      3 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price ASC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "Can you give me places where I can get Kadai Paneer and two naan for a low price? Veg only please.",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(bp:Food)
    WHERE bp.type = "veg" AND
          toLower(bp.name) =~ '(^|[^a-zA-Z])kadai paneer([^a-zA-Z]|$)'

    MATCH (r)-[:DELIVERS]->(n1:Food)
    WHERE toLower(n1.name) =~ '^(\w+\s)?(\w+\s)?(\w+\s)?naan$'

    MATCH (r)-[:DELIVERS]->(n2:Food)
    WHERE toLower(n2.name) =~ '^(\w+\s)?(\w+\s)?(\w+\s)?naan$' AND n1.id < n2.id

    WITH r.name AS restaurant,
        r.url as zomato_page,
        bp.name AS main_dish,
        n1.name AS naan_1,
        n2.name AS naan_2,
        round(bp.price + n1.price + n2.price, 2) AS total_price

    ORDER BY total_price ASC

    RETURN restaurant, main_dish, naan_1, naan_2, total_price, zomato_page
    '''
  },
  {
      "question": "Show me top-rated restaurants that list juices in their menu or deliverables.",
      "query": '''
      MATCH (r:Restaurant)
      WHERE toLower(r.deliverables) CONTAINS "juices"
        AND r.delivery_rating IS NOT NULL
        AND r.delivery_rating <> "not_available"
      WITH r, toFloat(r.delivery_rating) AS delivery_rating
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url AS zomato_page,
      r.image_url AS restaurant_image_url,
      CASE
        WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
        ELSE NULL
      END AS delivery_rating,
      CASE
        WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
        ELSE NULL
      END AS dining_rating,
      r.deliverables AS deliverables,
      r.phone_no AS phone_number,
      r.address AS address
      ORDER BY COALESCE(toFloatOrNull(delivery_rating), 0) DESC
      '''
  },
  {
      "question": "Hey, Are there any good italian restaurants, don't show me spanto restaurant?",
      "query": '''
      MATCH (r:Restaurant)
      WHERE toLower(r.deliverables) CONTAINS "italian"
        AND r.delivery_rating IS NOT NULL
        AND r.delivery_rating <> "not_available"
        AND NOT toLower(r.name) CONTAINS "spanto"
      WITH r, toFloat(r.delivery_rating) AS delivery_rating
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url AS zomato_page,
      r.image_url AS restaurant_image_url,
      CASE
        WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
        ELSE NULL
      END AS delivery_rating,
      CASE
        WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
        ELSE NULL
      END AS dining_rating,
      r.deliverables AS deliverables,
      r.phone_no AS phone_number,
      r.address AS address
      ORDER BY COALESCE(toFloatOrNull(delivery_rating), 0) DESC
      '''
  },
  {
      "question": "Hey, what are the options for juice available at Sunny's Juices & Shakes?",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
      WHERE (toLower(r.name) CONTAINS "sunny's juices & shakes") AND (toLower(f.name) CONTAINS "juice" OR toLower(f.desc) CONTAINS "juice")
      RETURN DISTINCT r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price ASC, COALESCE(toFloatOrNull(food_rating), 0) DESC
      '''
  },
  {
      "question": "Which is the most expensive pizza you have at dominos, ordering for 2 persons.",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)-[:COMES_UNDER]->(c:Category)
      WHERE (toLower(f.name) CONTAINS "pizza" OR toLower(f.desc) CONTAINS "pizza")
      AND (toLower(r.name) CONTAINS "dominos")
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      2 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
      LIMIT 1
      '''
  },
  {
      "question": "Show all the bestselling food items at zomato.",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
      WHERE f.bestseller = true
      OPTIONAL MATCH (f)-[:COMES_UNDER]->(c:Category)
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
      '''
  },
  {
      "question": "Can you show popular menu items from well-rated restaurants?",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
      WHERE f.bestseller = true
        AND r.delivery_rating IS NOT NULL
        AND r.delivery_rating <> 'not_available'
        AND r.delivery_rating <> 'non_available'
        AND toFloat(r.delivery_rating) > 2.0
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
      '''
  },
  {
      "question": "Give me bestseller food options where the restaurant has a decent delivery rating.",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
      WHERE f.bestseller = true
        AND r.delivery_rating IS NOT NULL
        AND r.delivery_rating <> 'not_available'
        AND r.delivery_rating <> 'non_available'
        AND toFloat(r.delivery_rating) > 2.0
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
      '''
  },
  {
      "question": "Give me combos of thali and basic lassi options at restaurants.",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(thali:Food)
      WHERE toLower(thali.name) CONTAINS "thali"

      MATCH (r)-[:DELIVERS]->(lassi:Food)
      WHERE toLower(lassi.name) =~ '^lassi.*'  // ensures "lassi" is at the beginning

      WITH r, thali, lassi
      RETURN
        r.name AS restaurant,
        r.url AS zomato_page,

        thali.name AS thali_name,
        thali.price AS thali_price,
        CASE
          WHEN thali.rating IS NOT NULL AND thali.rating <> 'not_available' THEN thali.rating
          ELSE NULL
        END AS thali_rating,
        thali.desc AS thali_description,
        thali.image_url AS thali_image,
        "Thali" AS thali_category,

        lassi.name AS lassi_name,
        lassi.price AS lassi_price,
        CASE
          WHEN lassi.rating IS NOT NULL AND lassi.rating <> 'not_available' THEN lassi.rating
          ELSE NULL
        END AS lassi_rating,
        lassi.desc AS lassi_description,
        lassi.image_url AS lassi_image,
        "Lassi" AS lassi_category
      '''
  },
  {
    "question": "I want plain lassi without any flavor",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
    WHERE toLower(f.name) =~ '^lassi.*'
      AND NOT toLower(f.name) CONTAINS "mango"
      AND NOT toLower(f.name) CONTAINS "sweet"
      AND NOT toLower(f.name) CONTAINS "flavored"
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
    "question": "Looking for roti without butter",
    "query": '''
    MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
    WHERE toLower(f.name) CONTAINS "roti"
      AND NOT toLower(f.name) CONTAINS "butter"
    RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url  AS zomato_page,
      r.delivery_rating AS delivery_rating,
      f.name AS food_name,
      f.bestseller AS bestseller,
      f.price AS price,
      f.type AS food_type,
      1 AS quantity,
      CASE
        WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
        ELSE NULL
      END AS food_rating,
      f.desc AS description,
      f.image_url AS food_image_url
      ORDER BY f.price DESC, COALESCE(toFloatOrNull(food_rating), 0) DESC
    '''
  },
  {
      "question": "What all restaurants are available near connaught place?",
      "query": '''
      MATCH (r:Restaurant)
      WHERE toLower(r.address) CONTAINS "connaught place"
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url AS zomato_page,
      r.image_url AS restaurant_image_url,
      CASE
        WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
        ELSE NULL
      END AS delivery_rating,
      CASE
        WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
        ELSE NULL
      END AS dining_rating,
      r.deliverables AS deliverables,
      r.phone_no AS phone_number,
      r.address AS address
      ORDER BY COALESCE(toFloatOrNull(delivery_rating), 0) DESC
      '''
  },
  {
      "question": "What all restaurants are available near connaught place which serves italian as well?",
      "query": '''
      MATCH (r:Restaurant)
      WHERE toLower(r.address) CONTAINS "connaught place"
      AND toLower(r.deliverables) CONTAINS "italian"
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url AS zomato_page,
      r.image_url AS restaurant_image_url,
      CASE
        WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
        ELSE NULL
      END AS delivery_rating,
      CASE
        WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
        ELSE NULL
      END AS dining_rating,
      r.deliverables AS deliverables,
      r.phone_no AS phone_number,
      r.address AS address
      ORDER BY COALESCE(toFloatOrNull(delivery_rating), 0) DESC
      '''
  },
  {
      "question": "Hey, can you confirm which restaurant has 1143595672 as its phone number?",
      "query": '''
      MATCH (r:Restaurant)
      WHERE toLower(r.phone_no) CONTAINS "1143595672"
      RETURN r.id AS restaurant_id,
      r.name AS restaurant,
      r.url AS zomato_page,
      r.image_url AS restaurant_image_url,
      CASE
        WHEN r.delivery_rating IS NOT NULL AND r.delivery_rating <> 'not_available' THEN r.delivery_rating
        ELSE NULL
      END AS delivery_rating,
      CASE
        WHEN r.dining_rating IS NOT NULL AND r.dining_rating <> 'not_available' THEN r.dining_rating
        ELSE NULL
      END AS dining_rating,
      r.deliverables AS deliverables,
      r.phone_no AS phone_number,
      r.address AS address
      ORDER BY COALESCE(toFloatOrNull(delivery_rating), 0) DESC
      '''
  },
  {
      "question": "Hey, which is the worst rated dish you have at zomato?",
      "query": '''
      MATCH (r:Restaurant)-[:DELIVERS]->(f:Food)
      WHERE f.rating IS NOT NULL AND f.rating <> "not_available"
      RETURN r.id AS restaurant_id,
        r.name AS restaurant,
        r.url  AS zomato_page,
        r.delivery_rating AS delivery_rating,
        f.name AS food_name,
        f.bestseller AS bestseller,
        f.price AS price,
        f.type AS food_type,
        1 AS quantity,
        CASE
          WHEN f.rating IS NOT NULL AND f.rating <> 'not_available' THEN f.rating
          ELSE NULL
        END AS food_rating,
        f.desc AS description,
        f.image_url AS food_image_url
      ORDER BY toFloat(f.rating) ASC
      LIMIT 1
      '''
  }
]

### Generate Cypher

In [89]:
# @title
import os
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_neo4j import Neo4jVector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embedding_model,
    Neo4jVector,
    k=5,
    input_keys=["question"]
)

text2cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "Given an input question, convert it to a Cypher query. No pre-amble."
                "Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"
            ),
        ),
        (
            "human",
            (
                """You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.

FOLLOWING TWO POINTS ARE VERY IMPORTANT:

1. Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only! Be consistent with the return result.

2. If the question demand food listing then always ensure that the cypher query must always return retaurant, zomato_page, food_name, bestseller, price, food_type, food_rating, description, food_image_url as present in the examples.

3. If you require to use WHERE clause for restaurant name i.e., r.name in cypher query, then always use like this way: toLower(r.name) CONTAINS <name_in_lowercase> OR NOT toLower(r.name) CONTAINS <name_in_lowercase>

Here is the schema information
{schema}

Below are a number of examples of questions and their corresponding Cypher queries.

{fewshot_examples}

User input: {question}
Cypher query:"""
            ),
        ),
    ]
)

text2cypher_chain = text2cypher_prompt | llm | StrOutputParser()

async def generate_cypher(state: OverallState) -> OverallState:
    """
    Generates a cypher statement based on the provided schema and user input
    """

    NL = '\n'
    fewshot_examples = (NL*2).join(
        [
            f"Question: {el['question']}{NL}Cypher:{el['query']}"
            for el in example_selector.select_examples(
                {"question": state.get("question")}
            )
        ]
    )

    generated_cypher = await text2cypher_chain.ainvoke(
        {
            "question": state.get("question"),
            "fewshot_examples": fewshot_examples,
            "schema": graph_schema,
        }
    )

    return {"cypher_statement": generated_cypher, "steps": ["generate_cypher"]}

### Validate Cypher

In [90]:
# @title
from neo4j.exceptions import CypherSyntaxError
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_neo4j.chains.graph_qa.cypher_utils import CypherQueryCorrector, Schema


validate_cypher_system = """
You are a Cypher expert reviewing a statement written by a junior developer.
"""

validate_cypher_user = """You must check the following:
* Are there any syntax errors in the Cypher statement?
* Are there any missing or undefined variables in the Cypher statement?
* Are any node labels missing from the schema?
* Are any relationship types missing from the schema?
* Are any of the properties not included in the schema?
* Does the Cypher statement include enough information to answer the question?

Examples of good errors:
* Label (:Foo) does not exist, did you mean (:Bar)?
* Property bar does not exist for label Foo, did you mean baz?
* Relationship FOO does not exist, did you mean FOO_BAR?

Schema:
{schema}

The question is:
{question}

The Cypher statement is:
{cypher}

Make sure you don't make any mistakes!"""

validate_cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            validate_cypher_system,
        ),
        (
            "human",
            (validate_cypher_user),
        ),
    ]
)

class Property(BaseModel):
    """
    Represents a filter condition based on a specific node property in a graph in a Cypher statement.
    """

    node_label: str = Field(
        description="The label of the node to which this property belongs."
    )

    property_key: str = Field(
        description="The key of the property being filtered."
    )

    property_value: str = Field(
        description="The value that the property is being matched against."
    )

class ValidateCypherOutput(BaseModel):
    """
    Represents the validation result of a Cypher query's output,
    including any errors and applied filters.
    """

    errors: Optional[List[str]] = Field(
        description="A list of syntax or semantical errors in the Cypher statement. Always explain the discrepancy between schema and Cypher statement"
    )

    filters: Optional[List[Property]] = Field(
        description="A list of property-based filters applied in the Cypher statement."
    )

validate_cypher_chain = validate_cypher_prompt | llm.with_structured_output(
    ValidateCypherOutput
)

# Cypher query corrector is experimental
corrector_schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph_structured_schema.get("relationships")
]

cypher_query_corrector = CypherQueryCorrector(corrector_schema)

async def validate_cypher(state: OverallState) -> OverallState:
    """
    Validates the Cypher statements and maps any property values to the database.
    """
    errors = []
    mapping_errors = []

    try:
    #   enhanced_graph.query(f"EXPLAIN {state.get('cypher_statement')}")
        await safe_query(query=f"EXPLAIN {state.get('cypher_statement')}")
    except CypherSyntaxError as e:
      errors.append(e.message)

    # Experimental feature for correcting relationship directions
    corrected_cypher = cypher_query_corrector(state.get("cypher_statement"))
    if not corrected_cypher:
      errors.append("The generated Cypher statement doesn't fit the graph schema")
    if not corrected_cypher == state.get("cypher_statement"):
      print("Relationship direction was corrected")

    # Use LLM to find additional potential errors and get the mapping for values

    llm_output = await validate_cypher_chain.ainvoke(
        {
            "question": state.get("question"),
            "cypher": state.get("cypher_statement"),
            "schema": graph_schema,
        }
    )

    if llm_output.errors:
      errors.extend(llm_output.errors)

    if llm_output.filters:
      for filter in llm_output.filters:
        if (
            not [
                prop
                for prop in graph_structured_schema["node_props"][
                    filter.node_label
                ]
                if prop["property"] == filter.property_key
            ][0]["type"]
            == "STRING"
        ):
          continue

        # mapping = enhanced_graph.query(
        #     f"MATCH (n: {filter.node_label}) WHERE toLower(n. `{filter.property_key}`) = toLower($value) RETURN 'yes' LIMIT 1",
        #     {"value": filter.property_value},
        # )

        mapping = await safe_query(
           query=f"MATCH (n: {filter.node_label}) WHERE toLower(n. `{filter.property_key}`) = toLower($value) RETURN 'yes' LIMIT 1",
           params={"value": filter.property_value},
        )

        if not mapping:
          print(
              f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
          )

          mapping_errors.append(
              f"Missing value mapping for {filter.node_label} on property {filter.property_key} with value {filter.property_value}"
          )

    if mapping_errors:
      next_action = "end"
    elif errors:
      next_action = "correct_cypher"
    else:
      next_action = "execute_cypher"


    return {
        "next_action": next_action,
        "cypher_statement": corrected_cypher,
        "cypher_errors": errors,
        "steps": ["validate_cypher"],
    }

### Correct Cypher

In [91]:
# @title
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

correct_cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a Cypher expert reviewing a statement written by a junior developer. "
                "You need to correct the Cypher statement based on the provided errors. No pre-amble."
                "Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"
            ),
        ),
        (
            "human",
            (
                """Check for invalid syntax or semantics and return a corrected Cypher statement.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not wrap the response in any backticks or anything else.
Respond with a Cypher statement only!

Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.

The question is:
{question}

The Cypher statement is:
{cypher}

The errors are:
{errors}

Corrected Cypher statement: """
            ),
        ),
    ]
)

correct_cypher_chain = correct_cypher_prompt | llm | StrOutputParser()

async def correct_cypher(state: OverallState) -> OverallState:
    """
    Correct the Cypher statement based on the provided errors.
    """

    corrected_cypher = await correct_cypher_chain.ainvoke(
        {
            "question": state.get("question"),
            "errors": state.get("cypher_errors"),
            "cypher": state.get("cypher_statement"),
            "schema": graph_schema,
        }
    )

    return {
        "next_action": "validate_cypher",
        "cypher_statement": corrected_cypher,
        "steps": ["correct_cypher"],
    }

### Execute Cypher

In [92]:
# @title
NO_RESULT = "I couldn't find any relevant information in the database"

async def execute_cypher(state: OverallState) -> OverallState:
    """
    Executes the given Cypher statement.
    """

    # records = enhanced_graph.query(state.get("cypher_statement"))
    records = await safe_query(query=state.get("cypher_statement"))

    return {
        "database_records": records if records else NO_RESULT,
        "next_action": "end",
        "steps": ["execute_cypher"],
    }

## Langgraph Agent

In [123]:
from langgraph.graph import END, START, StateGraph
from typing import Literal


def guardrails_condition(
    state: OverallState,
) -> Literal["generate_database_records", '__end__']:
  if state.get("next_action") == "end":
    # return "generate_final_answer"
    return END
  else: # next_action = "restaurant"
    return "generate_database_records"

def database_record_condition(
    state: OverallState,
) -> Literal["generate_cypher", '__end__']:
  if state.get("next_action") == "generate_cypher":
    return "generate_cypher"
  else: # next_action = "generate_final_answer"
    # return "generate_final_answer"
    return END

def validate_cypher_condition(
    state: OverallState
) -> Literal["correct_cypher", "execute_cypher", '__end__']:
  if state.get("next_action") == "correct_cypher":
    return "correct_cypher"
  elif state.get("next_action") == "execute_cypher":
    return "execute_cypher"
  else: # next_action == "end"
    return END


langgraph = StateGraph(state_schema=OverallState, input=InputState, output=OutputState)
langgraph.add_node(guardrails)
langgraph.add_node(generate_database_records)
langgraph.add_node(generate_cypher)
langgraph.add_node(validate_cypher)
langgraph.add_node(correct_cypher)
langgraph.add_node(execute_cypher)


langgraph.add_edge(START, "guardrails")
langgraph.add_conditional_edges(
    "guardrails",
    guardrails_condition,
)
langgraph.add_conditional_edges(
    "generate_database_records",
    database_record_condition,
)

langgraph.add_edge("generate_cypher", "validate_cypher")

langgraph.add_conditional_edges(
    "validate_cypher",
    validate_cypher_condition,
)

langgraph.add_edge("correct_cypher", "validate_cypher")
langgraph.add_edge("execute_cypher", END)

langgraph = langgraph.compile()


## Testing

In [136]:
question = """
what all italian restaurants are available at zomato near connaught place?
"""

question = """
Show me an option for white sauce past with rating atleast 2.5.
"""

question = """
Show me a good deal for a dal fry with roti under 300.
"""
question = "Mango Lassi, Kadai Paneer, Mix Veg"

question = """
Show me an option for cheese burger from KFC
"""

question = """
show me only the non-veg menu of KFC.
"""
question = "show me some options for dark chocolate cake."

question = "Which is highest rated chinese restaurant at zomato?"

question = "cheapest noodles option at zomato?"
question = """
My order: 1 - Kadai Paneer, 1 - Mix Veg, 6 - Roti or tandoori roti
"""

question = "What's the top 10 most expensive dish at zomato?"
question = """
Show me an option for top 10 rated cheese corn pizza under 300.
"""
question = "My order: 1 - Kadai Paneer, 1 - Mix Veg, 6 - Roti or tandoori roti, 2 - lassi, 3 - gulab jamun"
question = "show kfc restaurant"


In [137]:
question

'show kfc restaurant'

In [138]:
response = await langgraph.ainvoke({
    "question": question,
    "passing_threshold": 0.96
})

params:  {'name': [['kfc', True]]}
------------------------------------------------------------
Time spent in merging dataframe:  0.0
------------------------------------------------------------
Time spent in sorting dataframe:  0.0
------------------------------------------------------------
filter_flag:  False
Time spent in final output creation:  0.0
------------------------------------------------------------


In [139]:
len(response['database_records'])

1

In [140]:
response['database_records'][:10]

[{'deal': [{'restaurant': 'KFC',
    'zomato_page': 'https://www.zomato.com/ncr/kfc-3-paharganj-new-delhi/order',
    'restaurant_image_url': 'https://b.zmtcdn.com/data/pictures/chains/1/931/24697b617bb8aaf5b1c7df9a7074a662.jpg?fit=around|771.75:416.25&crop=771.75:416.25;*,*',
    'delivery_rating': '4.1',
    'dining_rating': '4.1',
    'deliverables': 'Burger, Rolls, Fast Food',
    'phone_number': '+918860172345',
    'address': 'Shop - 1, Ajmeri Gate, Paharganj, New Delhi'}]}]